In [1729]:
import pandas as pd
import mysql.connector

# Задача. Сделать классификатор по категориям рекламных роликов на основе их текстовых описаний.

# 1.. Формирование датасета из таблиц MySQL.

## 1..1.. Подключение к бд.

In [ ]:
# Реальные значения скрыты
DB_CONFIG = {
    'user': ***,
    'password': ***,
    'host': ***,
    'database': ***,
    'raise_on_warnings': False,
    'charset': 'utf8'
}

cnx = mysql.connector.connect(**DB_CONFIG)
cursor = cnx.cursor()

Описания полей таблицы external_creatives  
  
Таблица содержит информацию о рекламных креативах.  
Имеет очень большое количество полей,  
но нас интересуют только поля содержащие текстовую информацию:

- category_id - категория, к которой относится рекламный креатив. Целевая переменная. (Например Food & Drinks или Hobbies)
- title - заголовок рекламаного креатива, обычно это название рекламируемого продукта.
- description - описание рекламного ролика, более подробная информация о ролике и продукте.  
- adomain - домен рекламодателя, зачастую содержит название продукта.
- bundle - наименование бандла, также может содержать название продукта.
- vast - Video Ad Serving Template в формате xml, имеет подстроку AdTitle
- vast_origin - другой вариант васта, также имеет подстроку AdTitle

Описания полей таблицы categories 
   
В таблице id категорий соответствуют их коду и словесному названию  

- id
- parent_id - для подкатегорий, id родительской категории.
- iab_code - код категории по спецификации OpenRTB.  
- name - наименование категории.

In [181]:
query = ("SELECT * FROM categories LIMIT 5")

cursor.execute(query)
result = cursor.fetchall()
result

[(1, 0, 'IAB1', 'Arts & Entertainment'),
 (2, 1, 'IAB1-1', 'Books & Literature'),
 (3, 1, 'IAB1-2', 'Celebrity Fan/Gossip'),
 (4, 1, 'IAB1-3', 'Fine Art'),
 (5, 1, 'IAB1-4', 'Humor')]

## 1..2.. Анализ таблиц в бд.

Основные признаки, содержащие текстовую информацию -- это title и description.  
Также могут быть полезны adomain, bundle. И подстрока AdTitle у vast, vast_origin.

In [182]:
# Размер всего датасета
query = ("SELECT count(*) FROM external_creatives;")

cursor.execute(query)
result = cursor.fetchall()
result

[(618643,)]

In [183]:
# Количество строк, содержащих непустые title или description
query = ("SELECT count(*) FROM external_creatives WHERE title!='' OR description!='';")

cursor.execute(query)
result = cursor.fetchall()
result

[(279298,)]

In [184]:
# Количество строк c пустыми title или description, но непустыми adomain, bundle
query = ("SELECT count(*) FROM external_creatives WHERE title!='' OR description!=''"
         "OR bundle!='' OR adomain!='';")

cursor.execute(query)
result = cursor.fetchall()
result

[(381253,)]

In [186]:
# Количество строк c AdTitle, содержащими полезную информацию
query = ("SELECT count(*) FROM external_creatives WHERE vast not like "
         "'%<AdTitle>Linear Video Ad</AdTitle>%' OR vast_origin not like "
         "'%<AdTitle>Linear Video Ad</AdTitle>%';")

cursor.execute(query)
result = cursor.fetchall()
result

[(616260,)]

In [185]:
# Количество строк c непустой категорией. Интересуют именно такие.
query = ("SELECT COUNT(*) FROM external_creatives WHERE category_id!=0;")

cursor.execute(query)
result = cursor.fetchall()
result

[(171874,)]

In [187]:
query = ("SELECT COUNT(*)"
         "FROM external_creatives "
         "WHERE (title!='' OR description!='' OR bundle!='' OR adomain!='' "
         "OR vast not like '%<AdTitle>Linear Video Ad</AdTitle>%' "
         "OR vast_origin not like '%<AdTitle>Linear Video Ad</AdTitle>%') "
         "AND category_id!=0")

cursor.execute(query)
result = cursor.fetchall()
result

[(171870,)]

Получили, что практически все строки с непустой категорией содержат текстовую информацию. Однако, vast и vast_origin содержат и много ненужной информации.

In [188]:
query = ("SELECT COUNT(*)"
         "FROM external_creatives "
         "WHERE (title!='' OR description!='' OR bundle!='' OR adomain!='') "
         "AND category_id!=0")

cursor.execute(query)
result = cursor.fetchall()
result

[(171789,)]

Они составляют очень малую долю. Можно обойтись без vast и vast_origin.

In [189]:
query = ("SELECT COUNT(*)"
         "FROM external_creatives "
         "WHERE (title!='' OR description!='') "
         "AND category_id!=0")

cursor.execute(query)
result = cursor.fetchall()
result

[(144027,)]

Без bundle и domain не обойтись.

## 1..3.. Создание датасета.

In [191]:
query = ("SELECT id, title, description, adomain, bundle, category_id "
         "FROM external_creatives "
         "WHERE (title!='' OR description!='' OR bundle!='' OR adomain!='') "
         "AND category_id!=0")
sql_query = pd.read_sql_query (query, cnx)

df = pd.DataFrame(sql_query)

In [194]:
# Преобразуем данные чтобы отображалась кириллица.
for column in ['title', 'description']:
    df[column] = df[column].apply(lambda x: x.decode("utf-8", "strict"))

## Получили итоговый датасет.

In [196]:
df

,id,title,description,adomain,bundle,category_id
0,28,Приложения в Google Play Строки от МТС,"Книги, подкасты и аудиокниги",play.google.com,,149
1,9540091,Мы главные по качеству. Убедитесь в этом сами ...,Мы главные по качеству. Мы гарантируем каждому...,vkusnoitochka.ru,,1
2,27011430,Dodo Pizza - Apps on Google Play,Pizza delivery takeaway,play.google.com,,149
3,64518523,"Лента Онлайн доставка продуктов на дом, купить...",pageMetaDescription,online.lenta.com,,363
4,112454260,Samokat: Food Delivery on the AppStore,"Read reviews, compare customer ratings, see sc...",apps.apple.com,,294
...,...,...,...,...,...,...
171784,360777318394,Township on the AppStore,"Read reviews, compare customer ratings, see sc...",apps.apple.com,,294
171785,360777318395,Clockmaker: Mystery Match 3 on the AppStore,"Read reviews, compare customer ratings, see sc...",apps.apple.com,,294
171786,360777318396,Casas Bahia: Compras Online - Apps on Google Play,Want to WIN PIX to cheer for Brazil Its not a ...,play.google.com,,149
171787,360777318397,MISTPLAY: Play to earn rewards - Apps on Googl...,Collect units by playing games - exchange unit...,play.google.com,,149


In [197]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171789 entries, 0 to 171788
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           171789 non-null  int64 
 1   title        171789 non-null  object
 2   description  171789 non-null  object
 3   adomain      171789 non-null  object
 4   bundle       171789 non-null  object
 5   category_id  171789 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 7.9+ MB


In [198]:
# Выгрузим его в csv.
df.to_csv('df_origin.csv')

In [ ]:
# Также и категории выгрузим в датасет.
query = ("SELECT * FROM categories")
sql_query = pd.read_sql_query (query, cnx)

df_cats = pd.DataFrame(sql_query)

In [ ]:
cursor.close()
cnx.close()

In [ ]:
df_cats.to_csv('df_cats.csv')